### TAREA 1: Sistema de recuperación de noticias (RPP)

OBJETIVO:
Ingerir las últimas noticias de RPP (https://rpp.pe/rss), tokenizarlas, generar incrustaciones con SentenceTransformers,almacenarlas en ChromaDB y realizar consultas semánticas usando LangChain.

In [76]:
# Instalación segura y compatible con Google Colab o Jupyter
import sys
!{sys.executable} -m pip install --quiet \
    feedparser \
    tiktoken \
    sentence-transformers \
    chromadb \
    langchain-core \
    langchain-community \
    deep-translator \
    pandas \
    nltk

# Imports
import os
import feedparser
import pandas as pd
from datetime import datetime
from deep_translator import GoogleTranslator
from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt', quiet=True)

# Chroma y LangChain core (rutas correctas)
import chromadb
from chromadb.config import Settings
from langchain_core.runnables import RunnableLambda

print("✅ Librerías instaladas e importadas correctamente.")

✅ Librerías instaladas e importadas correctamente.


#### 0) Cargar y explorar el feed RSS de RPP

In [77]:
# Usamos feedparser para extraer las 50 noticias más recientes de RPP.
rss_url = "https://rpp.pe/rss"
feed = feedparser.parse(rss_url)
entries = feed.entries[:50]

data = []
for entry in entries:
    data.append({
        "title": entry.title,
        "description": getattr(entry, "summary", ""),   # some entries may lack summary
        "link": entry.link,
        "date_published": getattr(entry, "published", "")
    })

df = pd.DataFrame(data)
print(f"✅ Cargadas {len(df)} noticias desde: {rss_url}")
df.head()

✅ Cargadas 50 noticias desde: https://rpp.pe/rss


,title,description,link,date_published
0,Universitario vs. Sporting Cristal EN VIVO vía...,Sporting Cristal recibe a Universitario en un ...,https://rpp.pe/futbol/descentralizado/universi...,"Thu, 23 Oct 2025 20:05:08 -0500"
1,¡Empate agónico en Santiago! U. de Chile igual...,Lanús no aguantó la ventaja de 0-2 que tenía a...,https://rpp.pe/futbol/copa-sudamericana/u-de-c...,"Thu, 23 Oct 2025 19:35:16 -0500"
2,LDU Quito vs. Palmeiras EN VIVO vía ESPN: se e...,Consulta todos los detalles de lo que será el ...,https://rpp.pe/futbol/copa-libertadores/ldu-qu...,"Thu, 23 Oct 2025 19:35:45 -0500"
3,"""El dolor de mi corazón es muy fuerte”: Carlos...","Con una tierna imagen en redes sociales, el ac...",https://rpp.pe/famosos/farandula/carlos-vilche...,"Thu, 23 Oct 2025 18:59:23 -0500"
4,MEF y el presidente Jerí revisan el Presupuest...,"La ministra de Economía y Finanzas, Denisse Mi...",https://rpp.pe/economia/economia/mef-y-el-pres...,"Thu, 23 Oct 2025 19:18:25 -0500"


In [78]:
# Traducción del título y descripción al inglés
translator = GoogleTranslator(source="auto", target="en")

# aplicar traducción de forma robusta (captura excepciones)
def safe_translate(text):
    if not isinstance(text, str) or text.strip()=="":
        return ""
    try:
        return translator.translate(text)
    except Exception:
        return text  # fallback: devolver original si falla

df["title_en"] = df["title"].apply(safe_translate)
df["description_en"] = df["description"].apply(safe_translate)

print("✅ Traducción completada correctamente (title_en, description_en).")
df.head()

✅ Traducción completada correctamente (title_en, description_en).


,title,description,link,date_published,title_en,description_en
0,Universitario vs. Sporting Cristal EN VIVO vía...,Sporting Cristal recibe a Universitario en un ...,https://rpp.pe/futbol/descentralizado/universi...,"Thu, 23 Oct 2025 20:05:08 -0500",University vs. Sporting Cristal LIVE via L1MAX...,Sporting Cristal receives Universitario in a v...
1,¡Empate agónico en Santiago! U. de Chile igual...,Lanús no aguantó la ventaja de 0-2 que tenía a...,https://rpp.pe/futbol/copa-sudamericana/u-de-c...,"Thu, 23 Oct 2025 19:35:16 -0500",Agonizing draw in Santiago! U. de Chile tied 2...,Lanús could not hold on to the 0-2 lead it had...
2,LDU Quito vs. Palmeiras EN VIVO vía ESPN: se e...,Consulta todos los detalles de lo que será el ...,https://rpp.pe/futbol/copa-libertadores/ldu-qu...,"Thu, 23 Oct 2025 19:35:45 -0500",LDU Quito vs. Palmeiras LIVE via ESPN: they fa...,Check all the details of what will be LDU Quit...
3,"""El dolor de mi corazón es muy fuerte”: Carlos...","Con una tierna imagen en redes sociales, el ac...",https://rpp.pe/famosos/farandula/carlos-vilche...,"Thu, 23 Oct 2025 18:59:23 -0500","""The pain in my heart is very strong"": Carlos ...","With a tender image on social networks, the co..."
4,MEF y el presidente Jerí revisan el Presupuest...,"La ministra de Economía y Finanzas, Denisse Mi...",https://rpp.pe/economia/economia/mef-y-el-pres...,"Thu, 23 Oct 2025 19:18:25 -0500",MEF and President Jerí review the 2026 Public ...,"The Minister of Economy and Finance, Denisse M..."


#### 1) Tokenización de un artículo

In [79]:
# Contamos cuántos tokens tiene un texto (útil para saber si un modelo necesita fragmentar el contenido).

import tiktoken

# Usamos el modelo base "cl100k_base" (compatible con modelos OpenAI)
enc = tiktoken.get_encoding("cl100k_base")

# Tomamos la primera descripción
sample_text = df["description_en"].iloc[0] if len(df)>0 else ""
num_tokens = len(enc.encode(sample_text)) if sample_text else 0

print(f"Ejemplo de noticia traducida (primer registro):\n{sample_text}\n")
print(f"🔹 Número de tokens (primer registro): {num_tokens}")

Ejemplo de noticia traducida (primer registro):
Sporting Cristal receives Universitario in a vital match for both of them for a pending matchday in Liga1 I bet you.

🔹 Número de tokens (primer registro): 28


#### 2) Generar Embeddings con Sentence Transformers

In [80]:
# Creamos representaciones numéricas de los textos (vectores) usando el modelo "all-MiniLM-L6-v2".

from sentence_transformers import SentenceTransformer

# Cargamos el modelo de embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

# Generamos embeddings para cada descripción traducida
embeddings = embedder.encode(df["description_en"].tolist(), show_progress_bar=True)

# convertir a listas (seguro para Chroma)
embeddings_list = [e.tolist() if hasattr(e, "tolist") else list(e) for e in embeddings]

print(f"✅ Embeddings generados: {len(embeddings_list)} vectores.")

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Embeddings generados: 50 vectores.


#### 3) Crear base vectorial con ChromaDB

In [81]:
import chromadb
from chromadb.config import Settings
import os

# Directorio donde se guardará la base vectorial
PERSIST_DIR = "./chroma_store"
os.makedirs(PERSIST_DIR, exist_ok=True)

# Inicializar cliente Chroma
client = chromadb.Client(Settings(persist_directory=PERSIST_DIR))

# Si ya existe una colección anterior con errores, la eliminamos
try:
    client.delete_collection("rpp_news")
    print("🗑️ Colección antigua eliminada.")
except Exception as e:
    print("ℹ️ No había colección previa o ya fue eliminada.")

# Crear colección nueva y limpia
collection = client.create_collection(name="rpp_news")

# Insertar nuevamente las noticias con ambos idiomas
collection.add(
    documents=df["description_en"].tolist(),  # embeddings basados en textos traducidos al inglés
    embeddings=embeddings,
    metadatas=[
        {
            "title": t,
            "description_es": d_es,   # descripción original
            "description_en": d_en,   # descripción traducida
            "link": l,
            "date_published": date
        }
        for t, d_es, d_en, l, date in zip(
            df["title"], df["description"], df["description_en"], df["link"], df["date_published"]
        )
    ],
    ids=[f"id_{i}" for i in range(len(df))]
)

print(f"✅ {len(df)} noticias insertadas correctamente en la base vectorial (ChromaDB).")

🗑️ Colección antigua eliminada.
✅ 50 noticias insertadas correctamente en la base vectorial (ChromaDB).


#### 4) Consulta de similitud de noticia traducida

In [82]:
# Buscar noticias similares a la consulta
query = "Últimas noticias de economía"

# Traducimos la consulta al inglés
query_en = GoogleTranslator(source="auto", target="en").translate(query)

# Convertimos la consulta en embedding
query_embedding = embedder.encode([query_en])

# Recuperar los 5 resultados más similares
results = collection.query(query_embeddings=query_embedding, n_results=5)

# Convertir resultados en DataFrame
query_results = pd.DataFrame({
    "title": [m["title"] for m in results["metadatas"][0]],
    "description_es": [m["description_es"] for m in results["metadatas"][0]],
    "link": [m["link"] for m in results["metadatas"][0]],
    "date_published": [m["date_published"] for m in results["metadatas"][0]],
    "score": results["distances"][0]
})

print(f"\n🧭 Consulta original: {query}")
print(f"🌐 Traducción usada en búsqueda: {query_en}\n")

display(query_results)


🧭 Consulta original: Últimas noticias de economía
🌐 Traducción usada en búsqueda: Latest economic news



,title,description_es,link,date_published,score
0,MEF y el presidente Jerí revisan el Presupuest...,"La ministra de Economía y Finanzas, Denisse Mi...",https://rpp.pe/economia/economia/mef-y-el-pres...,"Thu, 23 Oct 2025 19:18:25 -0500",1.372490
1,Argentina: dólar blue hoy a cuánto cotiza este...,"La cotización del dólar blue, hoy jueves 23 de...",https://rpp.pe/mundo/argentina/argentina-dolar...,"Thu, 23 Oct 2025 05:59:39 -0500",1.385436
2,Kikko Corporation proyecta un crecimiento de h...,"Ricardo Kamego, presidente de Kikko Corporatio...",https://rpp.pe/economia/economia/kikko-corpora...,"Thu, 23 Oct 2025 17:30:55 -0500",1.407617
3,"Mitsui Automotriz apunta vender 8,500 Toyota y...",Mitsui Automotriz proyecta un sólido crecimien...,https://rpp.pe/economia/economia/mitsui-automo...,"Thu, 23 Oct 2025 18:00:30 -0500",1.479637
4,"Temblor en Perú, hoy 23 de octubre: magnitud y...",Actualización EN VIVO del último sismo en Perú...,https://rpp.pe/lima/desastres-naturales/temblo...,"Wed, 22 Oct 2025 02:18:58 -0500",1.537023


#### 5) Orquestación con LangChain (Carga → Tokeniza → Incrusta → Almacena → Recupera)

In [90]:
# Lllamamos librerias
from langchain_core.runnables import RunnableLambda, RunnableSequence
from deep_translator import GoogleTranslator
import nltk
nltk.download('punkt', quiet=True)

# --- Paso 1: Cargar el feed RSS ---
def load_rss(url):
    feed = feedparser.parse(url)
    entries = [
        {
            "title": e.title,
            "description": e.description,
            "link": e.link,
            "date_published": e.published
        }
        for e in feed.entries
    ]
    return entries

# --- Paso 2: Traducir al inglés (para embeddings) ---
def translate_entries(entries):
    translator = GoogleTranslator(source="auto", target="en")
    for e in entries:
        e["description_en"] = translator.translate(e["description"])
    return entries

# --- Paso 3: Tokenizar ---
def tokenize_entries(entries):
    for e in entries:
        e["tokens"] = nltk.word_tokenize(e["description_en"])
    return entries

# --- Paso 4: Generar embeddings y guardar ---
def embed_and_store(entries):
    from sentence_transformers import SentenceTransformer
    import chromadb
    from chromadb.config import Settings
    import os

    embedder = SentenceTransformer("all-MiniLM-L6-v2")

    os.makedirs("./chroma_store", exist_ok=True)
    client = chromadb.Client(Settings(persist_directory="./chroma_store"))
    collection = client.get_or_create_collection(name="rpp_news")

    descriptions = [e["description_en"] for e in entries]
    embeddings = embedder.encode(descriptions)

    metadatas = [
        {
            "title": e["title"],
            "description_es": e["description"],
            "link": e["link"],
            "date_published": e["date_published"]
        }
        for e in entries
    ]

    collection.add(
        documents=descriptions,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=[f"id_{i}" for i in range(len(entries))]
    )
    return collection

# --- Paso 5: Recuperar noticias similares ---
def retrieve_news(collection, query):
    from sentence_transformers import SentenceTransformer
    from deep_translator import GoogleTranslator
    import pandas as pd

    embedder = SentenceTransformer("all-MiniLM-L6-v2")

    query_en = GoogleTranslator(source="auto", target="en").translate(query)
    query_embedding = embedder.encode([query_en])

    results = collection.query(query_embeddings=query_embedding, n_results=5)

    df_results = pd.DataFrame({
        "title": [m["title"] for m in results["metadatas"][0]],
        "description_es": [m["description_es"] for m in results["metadatas"][0]],
        "link": [m["link"] for m in results["metadatas"][0]],
        "date_published": [m["date_published"] for m in results["metadatas"][0]],
        "score": results["distances"][0]
    })
    return df_results

# --- ORQUESTACIÓN LANGCHAIN ---
load_rss_r = RunnableLambda(lambda _: load_rss("https://rpp.pe/rss"))
translate_r = RunnableLambda(translate_entries)
tokenize_r = RunnableLambda(tokenize_entries)
embed_r = RunnableLambda(embed_and_store)
retrieve_r = RunnableLambda(lambda collection: retrieve_news(collection, "noticias sobre economía"))

pipeline = load_rss_r | translate_r | tokenize_r | embed_r | retrieve_r

# --- Ejecutar pipeline completo ---
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

resultados = pipeline.invoke(None)

print("✅ Pipeline completado. Noticias similares encontradas:")
display(resultados)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


✅ Pipeline completado. Noticias similares encontradas:


,title,description_es,link,date_published,score
0,"Isabelle Tate, actriz de '9-1-1: Nashville', m...",Su agencia confirmó la noticia en redes social...,https://rpp.pe/famosos/celebridades/murio-isab...,"Thu, 23 Oct 2025 17:00:54 -0500",1.363913
1,MEF y el presidente Jerí revisan el Presupuest...,"La ministra de Economía y Finanzas, Denisse Mi...",https://rpp.pe/economia/economia/mef-y-el-pres...,"Thu, 23 Oct 2025 19:18:25 -0500",1.448405
2,Argentina: dólar blue hoy a cuánto cotiza este...,"La cotización del dólar blue, hoy jueves 23 de...",https://rpp.pe/mundo/argentina/argentina-dolar...,"Thu, 23 Oct 2025 05:59:39 -0500",1.455216
3,Víctor Cutipa cambia de opinión y convoca a de...,El presidente de la Comisión de Energía y Mina...,https://rpp.pe/politica/congreso/victor-cutipa...,"Thu, 23 Oct 2025 16:25:52 -0500",1.540962
4,Kikko Corporation proyecta un crecimiento de h...,"Ricardo Kamego, presidente de Kikko Corporatio...",https://rpp.pe/economia/economia/kikko-corpora...,"Thu, 23 Oct 2025 17:30:55 -0500",1.555095


In [91]:
# Guardado de 50 noticias para atrea adicional
import os
import json

# Crear carpeta si no existe
os.makedirs("data", exist_ok=True)

# Cargar 50 noticias
entries = load_rss("https://rpp.pe/rss")[:50]

# Guardar JSON dentro de data/
with open("data/rpp_50.json", "w", encoding="utf-8") as f:
    json.dump(entries, f, ensure_ascii=False, indent=2)

print("✅ rpp_50.json guardado en la carpeta data/")


✅ rpp_50.json guardado en la carpeta data/
